In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor

import tensorflow as tf
tf.config.run_functions_eagerly(True)

from keras.utils import set_random_seed

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from keras.layers import * 
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping

2023-05-22 11:07:12.328494: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
nan_cols = test.isna().sum()
test1 = test.drop(nan_cols[nan_cols > 269].index.tolist(), axis = 1)
train1 = train.drop(nan_cols[nan_cols > 269].index.tolist(), axis = 1)

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)

# fit and transform the data
test_imputed = pd.DataFrame(imputer.fit_transform(test1), columns=test1.columns)
train_imputed = pd.DataFrame(imputer.fit_transform(train1), columns=train1.columns)

corr_matrix = train_imputed.corr().abs()

corr_y = train_imputed.corrwith(train_imputed['y'])
train_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1, inplace = True)
test_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1, inplace = True)

# identify predictors that have correlation of > 0.85
indices = np.where(np.triu(corr_matrix > 0.85, k=1))

# print the pairs of columns that have a correlation > 0.85
high_corr = []
count = 0
for idx in range(len(indices[0])):
    col1 = corr_matrix.columns[indices[0][idx]]
    col2 = corr_matrix.columns[indices[1][idx]]
    high_corr.append(col1)
    count+=1
    #print(f"{col1} and {col2}")
#print(count)

unique_vals = set(high_corr)
train_imputed = train_imputed.drop(unique_vals, axis = 1)

test_col = train_imputed.drop('y', axis = 1)
test_imputed = test_imputed[test_col.columns]

corr_matrix = train_imputed.corr().abs()

corr_y = train_imputed.corrwith(train_imputed['y'])

low_corr_cols = corr_y[corr_y < 0.01].index.tolist()

train_imputed = train_imputed.drop(low_corr_cols, axis = 1)
test_col = train_imputed.drop('y', axis = 1)
test_imputed = test_imputed[test_col.columns]

X = train_imputed.drop('y', axis = 1)
y = train_imputed.y

X_test = test_imputed

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

from sklearn.decomposition import PCA
plt.rcParams.update(plt.rcParamsDefault)
pca = PCA()
pca.fit(X_scaled)

pca = PCA(n_components = 0.99)
pca.fit(X_scaled)

X_train_pca = pca.transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

### Creating a FFNN 

In [4]:
model = Sequential()
model.add(Input(shape=(X_train_pca.shape[1],))) # input
model.add(Dropout(0.3))
model.add(Dense(500, activation='relu', kernel_initializer='he_normal')) 
model.add(Dropout(0.3))
model.add(Dense(200, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.3))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.3))
model.add(Dense(20, activation='relu', kernel_initializer='he_normal'))

model.add(Dense(1, activation='softmax'))

In [6]:
model.compile(loss='mean_absolute_error', optimizer='adam')

In [8]:
early = EarlyStopping('val_loss', patience=10)
set_random_seed(0)
history = model.fit(X_train_pca, y, epochs=100, validation_split=0.1, callbacks=early, batch_size=100)

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/100
49/49 [==============================] - 4s 72ms/step - loss: 7.6285 - val_loss: 7.6468
Epoch 2/100
49/49 [==============================] - 2s 47ms/step - loss: 7.6285 - val_loss: 7.6468
Epoch 3/100
49/49 [==============================] - 2s 48ms/step - loss: 7.6285 - val_loss: 7.6468
Epoch 4/100
49/49 [==============================] - 3s 51ms/step - loss: 7.6285 - val_loss: 7.6468
Epoch 5/100
49/49 [==============================] - 3s 56ms/step - loss: 7.6285 - val_loss: 7.6468
Epoch 6/100
49/49 [==============================] - 2s 42ms/step - loss: 7.6285 - val_loss: 7.6468
Epoch 7/100
49/49 [==============================] - 3s 60ms/step - loss: 7.6285 - val_loss: 7.6468
Epoch 8/100
49/49 [==============================] - 3s 59ms/step - loss: 7.6285 - val_loss: 7.6468
Epoch 9/100
49/49 [==============================] - 2s 39ms/step - loss: 7.6285 - val_loss: 7.6468
Epoch 10/100
49/49 [==============================] - 2s 47ms/step - loss: 7.6285 - val_loss: 7.6468

In [9]:
pred = pd.DataFrame()
pred['id'] = test['id']
pred['y'] = model.predict(X_test_pca)
pred = pred.set_index(['id'])
pred.to_csv('Witarsa_Ashley_regression.csv') # 13ish RMSE so shit 

 12/138 [=>............................] - ETA: 1s

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


138/138 [==============================] - 1s 8ms/step
